In [159]:
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import plotly.subplots as sp

In [160]:
DATAPATH2023 = 'data/original_data/weather_2023.CSV'
DATAPATH2024 = 'data/original_data/weather_2024.CSV'
DATACLEANED =  'data/preprocessed_data/water_consumption_cleaned_0.parquet'

In [161]:
COLUMN_MAPPING = {
    'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)': 'total_precip_mm',
    'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)': 'station_pressure_mb',
    'PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)': 'max_pressure_last_hour_mb',
    'PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)': 'min_pressure_last_hour_mb',
    'RADIACAO GLOBAL (Kj/m²)': 'global_radiation_kj_m2',
    'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)': 'air_temp_c',
    'TEMPERATURA DO PONTO DE ORVALHO (°C)': 'dew_point_temp_c',
    'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)': 'max_temp_last_hour_c',
    'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)': 'min_temp_last_hour_c',
    'TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)': 'max_dew_point_last_hour_c',
    'TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)': 'min_dew_point_last_hour_c',
    'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)': 'max_humidity_last_hour_percentage',
    'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)': 'min_humidity_last_hour_percentage',
    'UMIDADE RELATIVA DO AR, HORARIA (%)': 'relative_humidity_percentage',
    'VENTO, DIREÇÃO HORARIA (gr) (° (gr))': 'wind_direction_deg',
    'VENTO, RAJADA MAXIMA (m/s)': 'max_wind_gust_m_s',
    'VENTO, VELOCIDADE HORARIA (m/s)': 'wind_speed_m_s'
}


In [162]:
def preProcess(path):
    df = pd.read_csv(path, delimiter=';', skiprows=8, encoding='latin1', decimal=',')
    df.drop(df.columns[-1], axis=1, inplace=True) # Last column is empty due to ";" at the end of each line
    df['Hora UTC'] = df['Hora UTC'].apply(lambda x: datetime.strptime(x, '%H%M %Z')) 
    df['Data'] = pd.to_datetime(df['Data'], format='%Y/%m/%d')
    df['hour'] = df['Hora UTC'].dt.hour
    df['day'] = df['Data'].dt.day
    df['month'] = df['Data'].dt.month
    df['year'] = df['Data'].dt.year
    df.rename(columns=COLUMN_MAPPING, inplace=True)
    df.drop(columns=['Data', 'Hora UTC'], axis=1, inplace=True)

    return df

In [163]:
df23, df24 = preProcess(DATAPATH2023), preProcess(DATAPATH2024)

In [164]:
df_23_24_concat = pd.concat([df23, df24], axis=0)

In [165]:
df_23_24_concat['year'].value_counts()

year
2023    8760
2024    1440
Name: count, dtype: int64

In [166]:
cleaned_df = pd.read_parquet(DATACLEANED) 

In [167]:
cleaned_df.head()

,timestamp,flow_in_l_s,reservoir_level_percentage,pressure_mca,gmb_1_is_on,gmb_2_is_on
0,2023-03-17 11:27:06,68.59,29.86,38.20,0,1
1,2023-03-17 12:28:56,66.05,35.86,38.20,0,1
2,2023-03-17 12:31:26,65.64,36.16,38.06,0,1
3,2023-03-17 12:33:56,65.64,36.50,38.03,0,1
4,2023-03-17 12:36:26,65.64,36.80,38.17,0,1


In [168]:
cleaned_df['timestamp'] = pd.to_datetime(cleaned_df['timestamp'], format='%Y-%m-%d %H:%M:%S')
cleaned_df['hour'] = cleaned_df['timestamp'].dt.hour
cleaned_df['day'] = cleaned_df['timestamp'].dt.day
cleaned_df['month'] = cleaned_df['timestamp'].dt.month
cleaned_df['year'] = cleaned_df['timestamp'].dt.year

In [169]:
cleaned_df.head()

,timestamp,flow_in_l_s,reservoir_level_percentage,pressure_mca,gmb_1_is_on,gmb_2_is_on,hour,day,month,year
0,2023-03-17 11:27:06,68.59,29.86,38.20,0,1,11,17,3,2023
1,2023-03-17 12:28:56,66.05,35.86,38.20,0,1,12,17,3,2023
2,2023-03-17 12:31:26,65.64,36.16,38.06,0,1,12,17,3,2023
3,2023-03-17 12:33:56,65.64,36.50,38.03,0,1,12,17,3,2023
4,2023-03-17 12:36:26,65.64,36.80,38.17,0,1,12,17,3,2023


In [170]:
merged_df = cleaned_df.merge(df_23_24_concat, on=['hour', 'day', 'year', 'month'], how='left') 
# There is no temperature data available in some dates for the og data.

In [171]:
merged_df.head()

,timestamp,flow_in_l_s,reservoir_level_percentage,pressure_mca,gmb_1_is_on,gmb_2_is_on,hour,day,month,year,...,max_temp_last_hour_c,min_temp_last_hour_c,max_dew_point_last_hour_c,min_dew_point_last_hour_c,max_humidity_last_hour_percentage,min_humidity_last_hour_percentage,relative_humidity_percentage,wind_direction_deg,max_wind_gust_m_s,wind_speed_m_s
0,2023-03-17 11:27:06,68.59,29.86,38.20,0,1,11,17,3,2023,...,26.0,22.4,22.6,21.1,92.0,81.0,82.0,93.0,1.8,0.4
1,2023-03-17 12:28:56,66.05,35.86,38.20,0,1,12,17,3,2023,...,27.9,25.9,22.9,20.8,82.0,66.0,70.0,337.0,2.2,0.8
2,2023-03-17 12:31:26,65.64,36.16,38.06,0,1,12,17,3,2023,...,27.9,25.9,22.9,20.8,82.0,66.0,70.0,337.0,2.2,0.8
3,2023-03-17 12:33:56,65.64,36.50,38.03,0,1,12,17,3,2023,...,27.9,25.9,22.9,20.8,82.0,66.0,70.0,337.0,2.2,0.8
4,2023-03-17 12:36:26,65.64,36.80,38.17,0,1,12,17,3,2023,...,27.9,25.9,22.9,20.8,82.0,66.0,70.0,337.0,2.2,0.8


In [172]:
merged_df.isnull().sum()    

timestamp                               0
flow_in_l_s                             0
reservoir_level_percentage              0
pressure_mca                            0
gmb_1_is_on                             0
gmb_2_is_on                             0
hour                                    0
day                                     0
month                                   0
year                                    0
total_precip_mm                      3209
station_pressure_mb                  3209
max_pressure_last_hour_mb            3209
min_pressure_last_hour_mb            3209
global_radiation_kj_m2               3209
air_temp_c                           3209
dew_point_temp_c                     3209
max_temp_last_hour_c                 3209
min_temp_last_hour_c                 3209
max_dew_point_last_hour_c            3209
min_dew_point_last_hour_c            3209
max_humidity_last_hour_percentage    3209
min_humidity_last_hour_percentage    3209
relative_humidity_percentage      

In [173]:
print(f'merged_df shape: {merged_df.shape}\ncleaned_df shape: {cleaned_df.shape}\ndf_23_24_concat shape: {df_23_24_concat.shape}')

merged_df shape: (125073, 27)
cleaned_df shape: (125073, 10)
df_23_24_concat shape: (10200, 21)


In [174]:
nulls_df = merged_df[merged_df.isnull().any(axis=1)]
nulls_df.year.value_counts()

year
2024    3209
2023     166
Name: count, dtype: int64

In [175]:
nulls2023 = nulls_df[nulls_df['year'] == 2023]
nulls2023.shape

(166, 27)

In [176]:
nulls2024 = nulls_df[nulls_df['year'] == 2024]
nulls2024.shape

(3209, 27)

In [177]:
fig = sp.make_subplots(rows=2, cols=1)

trace1 = go.Scatter(x=nulls2023['day'], y=nulls2023['month'], mode='markers', name='nulls 2023')
trace2 = go.Scatter(x=nulls2024['day'], y=nulls2024['month'], mode='markers', name='nulls 2024')

fig.add_trace(trace1, row=1, col=1)
fig.add_trace(trace2, row=2, col=1)

fig.update_layout(height=600, width=600, title_text="Day and Month from nulls 2023 and nulls 2024")
fig.update_xaxes(title_text="Day", row=1, col=1)
fig.update_yaxes(title_text="Month", row=1, col=1)
fig.update_xaxes(title_text="Day", row=2, col=1)
fig.update_yaxes(title_text="Month", row=2, col=1)

fig.show()

### We do not have temperature data from 24/03/01 to 24/03/11 from the provided datasets

Found the complementary data but it is measured by "PORTO ALEGRE - JARDIM BOTANICO" station instead of "PORTO ALEGRE- BELEM NOVO" from the provided dataset

https://portal.inmet.gov.br/dadoshistoricos

In [178]:
DATAPATH2024_COMP = 'data/original_data/weather_2024_complementary.CSV'
comp_data = preProcess(DATAPATH2024_COMP)

In [179]:
filtered_comp_data = comp_data.query('1 <= day <= 11 and month == 3')


In [180]:

merged_df_indexed = merged_df.set_index(['day', 'month', 'hour', 'year'])
filtered_comp_data_indexed = filtered_comp_data.set_index(['day', 'month', 'hour', 'year'])

merged_df_indexed.update(filtered_comp_data_indexed)

full_df = merged_df_indexed.reset_index()

In [181]:
full_df.isnull().sum()

day                                     0
month                                   0
hour                                    0
year                                    0
timestamp                               0
flow_in_l_s                             0
reservoir_level_percentage              0
pressure_mca                            0
gmb_1_is_on                             0
gmb_2_is_on                             0
total_precip_mm                        57
station_pressure_mb                    57
max_pressure_last_hour_mb              57
min_pressure_last_hour_mb              57
global_radiation_kj_m2               1637
air_temp_c                             57
dew_point_temp_c                       57
max_temp_last_hour_c                   57
min_temp_last_hour_c                   57
max_dew_point_last_hour_c              57
min_dew_point_last_hour_c              57
max_humidity_last_hour_percentage      57
min_humidity_last_hour_percentage      57
relative_humidity_percentage      

In [182]:
full_df['global_radiation_kj_m2'] = full_df['global_radiation_kj_m2'].fillna(0)
full_df = full_df.ffill()

In [183]:
full_df.isnull().sum()

day                                  0
month                                0
hour                                 0
year                                 0
timestamp                            0
flow_in_l_s                          0
reservoir_level_percentage           0
pressure_mca                         0
gmb_1_is_on                          0
gmb_2_is_on                          0
total_precip_mm                      0
station_pressure_mb                  0
max_pressure_last_hour_mb            0
min_pressure_last_hour_mb            0
global_radiation_kj_m2               0
air_temp_c                           0
dew_point_temp_c                     0
max_temp_last_hour_c                 0
min_temp_last_hour_c                 0
max_dew_point_last_hour_c            0
min_dew_point_last_hour_c            0
max_humidity_last_hour_percentage    0
min_humidity_last_hour_percentage    0
relative_humidity_percentage         0
wind_direction_deg                   0
max_wind_gust_m_s        

In [184]:
full_df.drop(columns=['day', 'month', 'hour', 'year'], inplace=True)

In [185]:
full_df.to_parquet('data/preprocessed_data/water_consumption_merged_0.parquet')

In [186]:
full_df.head()

,timestamp,flow_in_l_s,reservoir_level_percentage,pressure_mca,gmb_1_is_on,gmb_2_is_on,total_precip_mm,station_pressure_mb,max_pressure_last_hour_mb,min_pressure_last_hour_mb,...,max_temp_last_hour_c,min_temp_last_hour_c,max_dew_point_last_hour_c,min_dew_point_last_hour_c,max_humidity_last_hour_percentage,min_humidity_last_hour_percentage,relative_humidity_percentage,wind_direction_deg,max_wind_gust_m_s,wind_speed_m_s
0,2023-03-17 11:27:06,68.59,29.86,38.20,0,1,0.0,1014.6,1014.6,1013.7,...,26.0,22.4,22.6,21.1,92.0,81.0,82.0,93.0,1.8,0.4
1,2023-03-17 12:28:56,66.05,35.86,38.20,0,1,0.0,1015.1,1015.1,1014.6,...,27.9,25.9,22.9,20.8,82.0,66.0,70.0,337.0,2.2,0.8
2,2023-03-17 12:31:26,65.64,36.16,38.06,0,1,0.0,1015.1,1015.1,1014.6,...,27.9,25.9,22.9,20.8,82.0,66.0,70.0,337.0,2.2,0.8
3,2023-03-17 12:33:56,65.64,36.50,38.03,0,1,0.0,1015.1,1015.1,1014.6,...,27.9,25.9,22.9,20.8,82.0,66.0,70.0,337.0,2.2,0.8
4,2023-03-17 12:36:26,65.64,36.80,38.17,0,1,0.0,1015.1,1015.1,1014.6,...,27.9,25.9,22.9,20.8,82.0,66.0,70.0,337.0,2.2,0.8
